Import dependencies and print Tensorflow version:

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.3.0'

In [2]:
INPUT_SIZE = 299
INPUT_CHANNELS_COUNT = 3
CLASSES_COUNT = 10
DATASET_LENGTH = 100
BATCH_SIZE = 32

def get_empty_image(i):
    image = tf.zeros([INPUT_SIZE, INPUT_SIZE, INPUT_CHANNELS_COUNT])
    one_hot_label = np.zeros([CLASSES_COUNT], np.bool)
    one_hot_label[0] = 1
    return image, one_hot_label

dataset = tf.data.Dataset.range(DATASET_LENGTH).map(get_empty_image).batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 299, 299, 3), (None, 10)), types: (tf.float32, tf.bool)>

Load the Saved Model, disable training of its first layer (which is the Xception model) and print its summary to verify, that it is similar to the original model:

In [3]:
H5_SAVE_PATH = "saved_model.h5"
saved_model_h5 = tf.keras.models.load_model(H5_SAVE_PATH)
saved_model_h5.layers[0].trainable = False
saved_model_h5.summary(line_length=120)

Model: "sequential"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
xception (Functional)                                 (None, 10, 10, 2048)                            20861480          
________________________________________________________________________________________________________________________
global_average_pooling2d (GlobalAveragePooling2D)     (None, 2048)                                    0                 
________________________________________________________________________________________________________________________
dense (Dense)                                         (None, 10)                                      20490             
Total params: 20,881,970
Trainable params: 20,490
Non-trainable params: 20,861,480
__________________________________________________

Now when trying to fit the Saved Model, we will get ResourceExhaustedError (out of memory):

In [4]:
H5_MODEL_PROFILING_LOG="profiling/h5_model"
with tf.profiler.experimental.Profile(H5_MODEL_PROFILING_LOG):
    saved_model_h5.fit(dataset, epochs=1)

4/4 [==============================] - 0s 122ms/step - loss: 1.4526


In [5]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [6]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir={H5_MODEL_PROFILING_LOG}